In [1]:
import os
import configparser
import io
import subprocess

from spatialist import crsConvert, Vector, Raster, bbox, intersect

In [2]:
##shp_fp, str, filepath to shapefile, must be a format Vector can read
##start, str, starting date of query, format %Y-%m-%d
##end, str, ending date of query, format %Y-%m-%d
##relOrbit, str, int or None, number of rel. orbit
##pol, str, selected polarization: ["VV", "VV+VH", "Dual VV", "Dual HV", "HH", "HH+HV", "VH", "Dual VH"]
##flightDir, str, ascending (A) and/or descending orbit (D)
##sensor, str, Sentinel-1A and/or Sentinel-1B
##procLvl, str, SLC or GRDH
##beamMode, str, selected beam mode of S-1
##nrpoc, int or str, number of parallel downloads
##cfg_name, str, filename of .config file
## verbose, bool, print output of query to console
##download, bool, initiate download of queried files 
def S1_query_ASF(shp_fp, start, end, relOrbit= None, pol= "VV+VH", flightDir= "ASC", sensor= "Sentinel-1A,Sentinel-1B", procLvl= "SLC", beamMode= "IW",\
                 nproc= "2", user= "username", password= "password",\
                 cfg_name= "S1_ASF_query.config", verbose= True, download = False):
    ##create wtk str from shp file
    shapefile= shp_fp
    if shapefile:
        # print('--- read')
        if isinstance(shapefile, dict):
            ext = shapefile
        else:
            if isinstance(shapefile, Vector):
                shp = shapefile.clone()
            elif isinstance(shapefile, str):
                shp = Vector(shapefile)
            else:
                raise TypeError("argument 'shapefile' must be either a dictionary, a Vector object or a string.")
            # reproject the geometry to WGS 84 latlon
            shp.reproject(4326)
            ext = shp.extent
            shp.close()
        # add an extra buffer of 0.01 degrees
        buffer = 0.01
        ext['xmin'] -= buffer
        ext['ymin'] -= buffer
        ext['xmax'] += buffer
        ext['ymax'] += buffer
        # print('--- create bbox')
        with bbox(ext, 4326) as bounds:
            # print('--- intersect')
            wkt = bounds.convert2wkt()[0]
    ##complete date strings with default time
    start= start+"T00:00:00UTC"
    end= end+ "T00:00:00UTC"
    ##int to str for rel orbit number and nproc
    if isinstance(relOrbit, int):
        relOrbit= str(relOrbit)
    
    if isinstance(nproc, int):
        nproc= str(nproc)
        
    pol_lookUp=["VV", "VV+VH", "Dual VV", "Dual HV", "HH", "HH+HV", "VH", "Dual VH"]
    
    ## create config file   
    configfile_name =  cfg_name
    # Check if there is already a configurtion file
    if not os.path.isfile(configfile_name):
        # Create the configuration file as it doesn't exist yet
        w= "w"
    elif os.path.isfile(configfile_name):
        w= "w+"
    cfgfile = open(configfile_name, w)

    # Add content to the file
    Config = configparser.ConfigParser()
    Config.add_section("api_search")
    Config.set("api_search", "output", "csv")
    Config.set("api_search", "platform", sensor)
    Config.set("api_search", "processingLevel", procLvl)
    Config.set("api_search", "beamMode", beamMode)
    Config.set("api_search", "intersectsWith", wkt)
    Config.set("api_search", "start", start)
    Config.set("api_search", "end", end)
    if relOrbit is not None: 
        Config.set("api_search", "relativeOrbit", relOrbit)
    if flightDir:
        Config.set("api_search", "flightDirection", flightDir)
    if pol:
        ##encode pol string acording to ASF specifications
        if pol in pol_lookUp:
            if "+" in pol:
                pol=pol.replace("+", "%%2B")
            elif " " in pol:
                pol=pol.replace(" ", "+")

            Config.set("api_search", "polarization", pol)
        else:
            raise RuntimeError("Selected polarization not available")

    Config.add_section("download")
    Config.set("download", "download_site", "ASF")
    Config.set("download", "nproc", nproc)

    Config.add_section("asf_download")
    Config.set("asf_download","http-user", user)
    Config.set("asf_download", "http-password", password)

    Config.write(cfgfile)
    cfgfile.close()

    ##hand over the query to shell and execute
    ##shell command according to specifications
    if verbose == True and download == True:
        cmd = ['python3', 'sentinel_query_download.py', configfile_name, '--verbose', "--download"]
    elif verbose == True and download == False:
        cmd = ['python3', 'sentinel_query_download.py', configfile_name, '--verbose']
    elif verbose == False and download == True:
        cmd = ['python3', 'sentinel_query_download.py', configfile_name, '--download']
    else:
        cmd = ['python3', 'sentinel_query_download.py', configfile_name]
        
    with subprocess.Popen(cmd, stdout=subprocess.PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='')
    

In [3]:
#S1_query_ASF(shp_fp= "BBox_fields_demmin.shp", start= "2019-03-01", end= "2019-06-01", user= "", password= "", download= True, flightDir= "DESC")
